<a href="https://colab.research.google.com/github/janaat22/NLP_NER/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
pip install sklearn-crfsuite

     |████████████████████████████████| 757kB 7.2MB/s 


In [0]:
#Data analysis
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn_crfsuite import CRF, scorers, metrics
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
#from sklearn.grid_search import RandomizedSearchCV
import scipy.stats
#import eli5

In [105]:
import pandas as pd
import numpy as np
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:

# Using readlines() 
file1 = open('train.txt', 'r') 
Lines = file1.readlines() 

training_key_value = []
word_dict = {}
tags = {}
word_list = []
sentence = 1
for line in Lines:    
    training_key_value = line.split()
    if(len(training_key_value) == 2):
      word = training_key_value[0]
      tag = training_key_value[1]
      word_dict[word] = tag
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      word_list.append(([sentence, word, tagged[0][1], tag])) 
      tags.update(tagged)
    else:
      sentence = sentence + 1


#print("test " + str(word_dict))
array_of_words = np.array(list(word_dict.keys()))
array_of_values = np.array(list(word_dict.values()))
X_Y = np.column_stack((array_of_words, array_of_values))




In [107]:
print("tags" + str(tags))
print("shape of X_Y " + str(X_Y.shape))
print("word_list" + str(word_list))

tags{'@': 'NN', 'SammieLynnsMom': 'NN', 'tg10781': 'NN', 'they': 'PRP', 'will': 'MD', 'be': 'VB', 'all': 'DT', 'done': 'VBN', 'by': 'IN', 'Sunday': 'NNP', 'trust': 'NN', 'me': 'PRP', '*wink*': 'NN', 'Made': 'VBN', 'it': 'PRP', 'back': 'RB', 'home': 'NN', 'to': 'TO', 'GA': 'NN', '.': '.', 'It': 'PRP', 'sucks': 'NNS', 'not': 'RB', 'at': 'IN', 'Disney': 'NN', 'world': 'NN', ',': ',', 'but': 'CC', 'its': 'PRP$', 'good': 'JJ', 'Time': 'NN', 'start': 'NN', 'planning': 'NN', 'the': 'DT', 'next': 'JJ', 'World': 'NN', 'trip': 'NN', "'": "''", 'Breaking': 'VBG', 'Dawn': 'NN', 'Returns': 'NNS', 'Vancouver': 'NNP', 'on': 'IN', 'January': 'NNP', '11th': 'CD', 'http': 'NN', ':': ':', '//bit.ly/dbDMs8': 'NN', 'ls_n': 'NN', 'perhaps': 'RB', 'folks': 'NNS', 'may': 'MD', 'find': 'VB', 'something': 'NN', 'in': 'IN', 'gallery': 'NN', 'that': 'IN', 'is': 'VBZ', 'helpful': 'NN', 'their': 'PRP$', 'day-to-day': 'NN', 'work': 'NN', 'as': 'IN', 'well': 'RB', 'Even': 'RB', 'just': 'RB', 'use': 'NN', 'Carr0t': 'N

In [108]:
data = pd.DataFrame(word_list)
print(data.head(n = 20))

    0                1    2           3
0   1  @SammieLynnsMom   NN           O
1   1         @tg10781   NN           O
2   1             they  PRP           O
3   1             will   MD           O
4   1               be   VB           O
5   1              all   DT           O
6   1             done  VBN           O
7   1               by   IN           O
8   1           Sunday  NNP           O
9   1            trust   NN           O
10  1               me  PRP           O
11  1           *wink*   NN           O
12  2             Made  VBN           O
13  2               it  PRP           O
14  2             back   RB           O
15  2             home   NN           O
16  2               to   TO           O
17  2               GA   NN  B-location
18  2                .    .           O
19  2               It  PRP           O


In [109]:
data.columns = data.iloc[0]
data = data[1:]
data.columns = ['Sentence','Word','POS','Tag']
data = data.reset_index(drop=True)
data.head(n=5)

,Sentence,Word,POS,Tag
0,1,@tg10781,NN,O
1,1,they,PRP,O
2,1,will,MD,O
3,1,be,VB,O
4,1,all,DT,O


In [110]:
data.shape

(46468, 4)

In [0]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [115]:
getter = getsentence(data)
sentences = getter.sentences
#This is how a sentence will look like. 
print(sentences[0])


[('@tg10781', 'NN', 'O'), ('they', 'PRP', 'O'), ('will', 'MD', 'O'), ('be', 'VB', 'O'), ('all', 'DT', 'O'), ('done', 'VBN', 'O'), ('by', 'IN', 'O'), ('Sunday', 'NNP', 'O'), ('trust', 'NN', 'O'), ('me', 'PRP', 'O'), ('*wink*', 'NN', 'O')]


In [0]:
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

In [0]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

In [0]:
#Creating the train and test set
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [0]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [130]:
#We predcit using the same 5 fold cross validation
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [131]:
#Lets evaluate the mode
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

   B-company       0.84      0.44      0.58       171
     B-group       0.63      0.18      0.28       106
  B-location       0.74      0.46      0.57       380
     B-other       0.52      0.24      0.33       225
    B-person       0.70      0.47      0.56       449
   B-product       0.83      0.15      0.26        97
     B-title       0.48      0.16      0.24        68
   I-company       0.43      0.08      0.14        36
     I-group       0.61      0.13      0.22        84
  I-location       0.64      0.38      0.48       154
     I-other       0.47      0.30      0.36       320
    I-person       0.70      0.56      0.62       215
   I-product       0.74      0.17      0.28        80
     I-title       0.35      0.14      0.20        77
           O       0.97      0.99      0.98     44006

    accuracy                           0.96     46468
   macro avg       0.64      0.32      0.41     46468
weighted avg       0.95   